In [59]:
import json
import requests
import pandas as pd
import numpy as np

In [60]:
#queries

deposits_query_template = """
{
  deposits {
    account {
      id
    }
    amount
    asset {
      id
      name
      symbol
      decimals
    }
    timestamp
    hash
  }
}
"""
withdraws_query = """
{
withdraws {
    account {
      id
    }
    amount
    asset {
      id
      name
      symbol
      decimals
    }
    timestamp
    hash
  }
}
"""

In [61]:
# pull raw Seamless (Aave v3 fork) data from goldsky API and insert into pandas dataframe for analysis

def pull_data(query):
    url = "https://api.goldsky.com/api/public/project_clvq2euuzbinl01vgenyc84zw/subgraphs/seamless/1.0.0/gn"
    response = requests.post(url, json={'query': query})
    data = response.json()
    try:
        response.raise_for_status() 
        data = response.json()
        return data
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as err:
        print(f"Other error occurred: {err}")
    return {}


deposits_response = pull_data(deposits_query)
withdraws_response = pull_data(withdraws_query)


deposits_data = deposits_response.get("data", {}).get("deposits", [])
withdraws_data = withdraws_response.get("data", {}).get("withdraws", [])


# Parse the data into a list of dictionaries
parsed_deposits = [
    {
        "transaction_type": "deposit",
        "account_id": deposit["account"]["id"],
        "amount": deposit["amount"],
        "asset_id": deposit["asset"]["id"],
        "asset_name": deposit["asset"]["name"],
        "asset_symbol": deposit["asset"]["symbol"],
        "asset_decimals": deposit["asset"]["decimals"],
        "timestamp": deposit["timestamp"],
        "hash": deposit["hash"],
    }
    for deposit in deposits_data if deposit  
]

parsed_withdraws = [
    {
        "transaction_type": "withdraw",
        "account_id": withdraw["account"]["id"],
        "amount": withdraw["amount"],
        "asset_id": withdraw["asset"]["id"],
        "asset_name": withdraw["asset"]["name"],
        "asset_symbol": withdraw["asset"]["symbol"],
        "asset_decimals": withdraw["asset"]["decimals"],
        "timestamp": withdraw["timestamp"],
        "hash": withdraw["hash"],
    }
    for withdraw in withdraws_data if withdraw 
]

# Combine the parsed data
combined_data = parsed_deposits + parsed_withdraws

combined_df = pd.DataFrame(combined_data)

combined_df.head()


,transaction_type,account_id,amount,asset_id,asset_name,asset_symbol,asset_decimals,timestamp,hash
0,deposit,0x4012732e2e2794153f3d1dba987745ff3537afc8,192954643598604256,0x4200000000000000000000000000000000000006,Wrapped Ether,WETH,18,1711935459,0x00004c58ff559588a0852d94446ffc74676980a32096...
1,deposit,0x22539dae0aa613512ed8642fce92a62eaa161f45,270662795074440288,0x4200000000000000000000000000000000000006,Wrapped Ether,WETH,18,1712989559,0x0000840f2fb39f5b2a2bd3caa352c7727f2b1ea54799...
2,deposit,0x515ec0ad96e91218dc799e72c1b91742f510a0ba,1000000000000000,0x4200000000000000000000000000000000000006,Wrapped Ether,WETH,18,1702504965,0x000228c6f88025b5d43375b0743e1df183532530b5a1...
3,deposit,0x836c78746a0cb114fe441eb70005b40e2c12a1ad,3000000,0xd9aaec86b65d86f6a7b5b1b0c42ffa531710b6ca,USD Base Coin,USDbC,6,1711956583,0x0002b0db4918432d5e5ffe0414625ca8aa6adfe4f084...
4,deposit,0x771c4150bc2bfe4df9632513c2948c2754911a4d,10000000,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,USD Coin,USDC,6,1718295653,0x000355b4b45570b893082af01778c4dee91c65469e08...


In [62]:
# adjust timestamp for readability
combined_df['timestamp'] = pd.to_numeric(combined_df['timestamp'])
combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], unit='s')
combined_df = combined_df.sort_values(by='timestamp', ascending=False)
combined_df['timestamp'] = combined_df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')


In [63]:
# adjust amount for decimals
combined_df['amount'] = pd.to_numeric(combined_df['amount'], errors='coerce')
combined_df['asset_decimals'] = pd.to_numeric(combined_df['asset_decimals'], errors='coerce')
combined_df['amount'] = combined_df['amount'] / 10 ** combined_df['asset_decimals']

combined_df.head()

,transaction_type,account_id,amount,asset_id,asset_name,asset_symbol,asset_decimals,timestamp,hash
4,deposit,0x771c4150bc2bfe4df9632513c2948c2754911a4d,10.000000,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,USD Coin,USDC,6,2024-06-13 16:20:53,0x000355b4b45570b893082af01778c4dee91c65469e08...
118,withdraw,0xaeeb3898ede6a6e86864688383e211132baa1af3,0.006001,0x4200000000000000000000000000000000000006,Wrapped Ether,WETH,18,2024-06-12 08:24:25,0x000a83d834f33b59463a4aba41ce1e83c0766dc75909...
127,withdraw,0xaeeb3898ede6a6e86864688383e211132baa1af3,0.002500,0x4200000000000000000000000000000000000006,Wrapped Ether,WETH,18,2024-06-12 06:46:57,0x000de8507c788d91684fe97659d642a774bf69eef28c...
20,deposit,0x37801d13f75b00bd97e7183ea5a6d0329b29be63,0.087451,0x4200000000000000000000000000000000000006,Wrapped Ether,WETH,18,2024-06-11 12:17:53,0x00096ab4606d91adbe1be06b27dd43e11a42965c7a73...
90,deposit,0xdef31baf58f7f4d9a77cb4cff40b343f8ca822e9,3.000000,0x833589fcd6edb6e08f4c7c32d4f71b54bda02913,USD Coin,USDC,6,2024-06-11 07:24:29,0x001bb2fd4bcbe070c17e272406f88680821a360b3f90...


In [64]:
combined_df.to_csv('seamless_data.csv', index=False)